# Monkeypox

Mit dem Skript lassen sich Daten für den Artikel zu Affenpocken manuell aufbereiten. Das automatisierte Skript ist auf Github unter st-methods/monkeypox abgelegt. <br>
LD-Artikel: https://edit.nzz.ch/p/nzz/articles/1685302/edit/canvas

Quelle Github: https://github.com/globaldothealth/monkeypox


In [20]:
import pandas as pd
import webbrowser
import pycountry
import gettext
from datetime import datetime
import os
import numpy as np

In [21]:
german = gettext.translation('iso3166', pycountry.LOCALES_DIR, languages=['de'])
german.install()

## DF einlesen

In [22]:
# read google sheet
sheet_id = '1CEBhao3rMe-qtCbAgJTn5ZKQMRFWeAeaiXFpBY3gbHE'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv"
df_roh = pd.read_csv(url)

In [23]:
# Pivot wider
df = df_roh.groupby(['Country','Status'])['ID'].count().unstack().reset_index()
df= df.rename(columns={'confirmed':'Bestätigt', 'suspected':'Verdacht'})

df.drop('discarded', inplace=True, axis=True)

# add totals column
df['Total']=df.iloc[:,1:].sum(axis=True)

df = df[df['Total']!=0].reset_index(drop = True)

In [24]:

# format integers
df = df.fillna(0)
df.iloc[:,1:] = df.iloc[:,1:].astype(int)
df['Country'] = df['Country'].str.strip()

In [25]:
# sum up UK and France
def sum_up_countries(df, subcountries, new_name):
    country_idx = df[df['Country'].isin(subcountries)].index
    country_row = df.iloc[country_idx].sum()
    country_row.loc['Country'] = new_name
    df =df.append(country_row, ignore_index=True)
    df.drop(country_idx, inplace=True)
    return df.reset_index(drop=True)
    
df = sum_up_countries(df, ['England', 'Scotland', 'Wales', 'Northern Ireland','United Kingdom'], 'United Kingdom' )
df = sum_up_countries(df, ['France', 'French Guiana'], 'France' )

In [26]:
df=df.sort_values('Total', ascending=False)

# replace country names with our worldmap ids
df['Country'] = df['Country'].str.replace('Czech Republic', 'Czechia')
df['Country'] = df['Country'].str.replace('Iran', 'Iran, Islamic Republic of')


In [27]:
# temporarily remove kosovo

df = df[df['Country']!='Kosovo']

In [28]:
ignore_lst_pycountry = ['Kosovo']

# Catch country name if not in pycountry
for name in df['Country'].to_list():
    if name in ignore_lst_pycountry:
        continue
    if pycountry.countries.get(name=name)== None and pycountry.countries.get(common_name =name) == None:
        #print(pycountry.countries.get(common_name =name).common_name)
        raise ValueError('Country is not recognised by pycountry:', name)

In [29]:
#df

### Deutsche Ländernamen, Flaggen

In [30]:
# get country object, using common name if there is one
def get_common_name(name):
    if pycountry.countries.get(common_name =name) == None: 
        country = pycountry.countries.get(name=name)
    else:
        country = pycountry.countries.get(common_name =name)
    return country

# return german name
def get_german_name(name):
    country = get_common_name(name)
    if pycountry.countries.get(common_name =name) == None:
        country = country.name
    else:
        country = country.common_name        
    return _(country)

# return flag
def get_flag(name):
    country = get_common_name(name)
    return country.flag


In [31]:
df['Land'] = df['Country'].apply(get_german_name)
df['Flagge'] =  df['Country'].apply(get_flag)

# Iran zurückwandeln
df['Country'] = df['Country'].str.replace('Iran, Islamic Republic of', 'Iran')
df['Country'] = df['Country'].str.replace('Iran, Islamic Republic of', 'Iran')

df = df[['Country', 'Land', 'Flagge','Bestätigt', 'Verdacht', 'Total']]

In [32]:
#df

## Vergleichen, Version abspeichern

In [33]:
past_csvs= os.listdir('data')

past_csvs.sort()
print(past_csvs)

['.gitkeep', '20220524-16h03.csv']


In [34]:
df_last_version = pd.read_csv('data/'+past_csvs[-1])

In [35]:
print('old == new:', str(df['Total'].sum() == df_last_version['Total'].sum()))
print('new total:', str(df['Total'].sum()))
print('old total:', str(df_last_version['Total'].sum()))

old == new: False
new total: 2121
old total: 270


In [37]:
# save most recent data
time_now = datetime.now().strftime("%Y%m%d-%Hh%M")

df.to_csv('data/'+ time_now+'.csv', index=False)

## Export für Weltkarte
https://q.st.nzz.ch/editor/choropleth/4acf1a0fd4dd89aef4abaeefd0b6f4dc

In [38]:
ids = pd.read_csv('country_ids.csv')

df_worldmap =df.copy()
#df_worldmap = df_worldmap[df_worldmap['Country']!='Malta']
df_worldmap['Country'] = df_worldmap['Country'].str.replace('United States','United States of America')
df_worldmap['Country'] = df_worldmap['Country'].str.replace('Bahamas','The Bahamas')
df_worldmap = df_worldmap.rename(columns ={'Country':'ID', 'Total':'Wert'})

# merge df with ids
df_worldmap = ids.merge(df_worldmap[['ID', 'Wert']], how='left').sort_values('Wert', ascending=False)

In [40]:
# Check if all countries were recognised - is True if ok

ignore_lst = ['Cayman Islands', 'Gibraltar']
total_df = df[~df['Country'].isin(ignore_lst)]['Total'].sum()

print('Sum Worldmap:', str(df_worldmap['Wert'].sum()))
#ok if it's not Malta
print('Sum DF:', str(total_df))
print('Corresponds with sum df:', str(df_worldmap['Wert'].sum() == total_df))

Sum Worldmap: 2119.0
Sum DF: 2119
Corresponds with sum df: True


In [41]:
# copy to q-element
df_worldmap=df_worldmap.sort_values('ID', key=lambda col: col.str.lower())

df_worldmap.to_clipboard(index=False)
#webbrowser.open('https://q.st.nzz.ch/editor/choropleth/4acf1a0fd4dd89aef4abaeefd0b6f4dc', new=2)

### Differenzen finden

In [42]:
df[df['Total']==1].sort_values('Country')


Status,Country,Land,Flagge,Bestätigt,Verdacht,Total
3,Bahamas,Bahamas,🇧🇸,0,1,1
5,Bolivia,Bolivien,🇧🇴,0,1,1
8,Cayman Islands,Cayman-Inseln,🇰🇾,0,1,1
11,Ecuador,Ecuador,🇪🇨,0,1,1
13,Georgia,Georgien,🇬🇪,1,0,1
16,Gibraltar,Gibraltar,🇬🇮,1,0,1
17,Greece,Griechenland,🇬🇷,1,0,1
18,Haiti,Haiti,🇭🇹,0,1,1
19,Hungary,Ungarn,🇭🇺,1,0,1
27,Luxembourg,Luxemburg,🇱🇺,1,0,1


In [43]:
df_worldmap[df_worldmap['Wert']==1].sort_values('ID')

,ID,Wert
16,Bolivia,1.0
42,Ecuador,1.0
55,Georgia,1.0
58,Greece,1.0
64,Haiti,1.0
67,Hungary,1.0
93,Luxembourg,1.0
98,Malta,1.0
102,Mexico,1.0
106,Morocco,1.0


## Export für Tabelle

https://q.st.nzz.ch/editor/table/4acf1a0fd4dd89aef4abaeefd0da5ac6

In [44]:
df_q_table = df[['Land', 'Flagge', 'Bestätigt', 'Verdacht','Total']].rename(columns = {'Land':'', 'Flagge':''})
#df_q_table

In [45]:
df_q_table.to_clipboard(index=False)

#webbrowser.open('https://q.st.nzz.ch/editor/table/4acf1a0fd4dd89aef4abaeefd0da5ac6')

## Zahlen für LD-Artikel

In [46]:
# Fälle total (inklusive inoffizielle Länder)
df['Total'].sum()

2121

## Export Geschlechtergrafik [inaktiv]
https://q.st.nzz.ch/editor/chart/4913f749b598fb2ecc9721cb17e708e9

In [47]:
gender_dict = {'Male':'Männer', 'male':'Männer',
              'female':'Frauen',np.nan:'Keine Daten'}

In [48]:
df_gender = df_roh.copy()
df_gender['Gender'] = df_gender['Gender'].replace(gender_dict)

# only group confirmed cases
df_q_gender = df_gender[df_gender['Status']=='confirmed'].groupby(['Gender'], dropna=False, as_index=False).size()
df_q_gender.rename(columns={'Gender':'Geschlecht', 'size':'Anzahl'})

df_q_gender = df_q_gender.T
df_q_gender.columns = df_q_gender.iloc[0]
df_q_gender.drop('Gender', inplace=True)
df_q_gender = df_q_gender[['Männer', 'Keine Daten', 'Frauen']]

In [49]:
df_q_gender.to_clipboard()
#webbrowser.open('https://q.st.nzz.ch/editor/chart/4913f749b598fb2ecc9721cb17e708e9')

In [50]:
df_q_gender

Gender,Männer,Keine Daten,Frauen
size,524,1493,3


In [ ]:
df_q_gender.sum(axis=True)

## Test: Zeitreihe

In [51]:

df_roh['Date_onset'] = pd.to_datetime(df_roh['Date_onset'])
df_roh['Date_confirmation'] = pd.to_datetime(df_roh['Date_confirmation'])
df_roh['Date_entry'] = pd.to_datetime(df_roh['Date_entry'])
df_roh['Date_last_modified'] = pd.to_datetime(df_roh['Date_last_modified'])

In [52]:
import altair as alt

### Rollender Durchschnitt

1. Summe pro Tag
2. Rollender Schnitt pro Woche

In [81]:
df_cases_per_day = df_roh.groupby(df_roh['Date_entry'].dt.date).size().reset_index()
df_cases_per_day.columns = ['Date_entry', 'Cases']
df_cases_per_day['Date_entry'] = pd.to_datetime(df_cases_per_day['Date_entry'], format='%Y-%m-%d')

In [82]:
df_cases_per_day['Rolling_mean'] =df_cases_per_day['Cases'].rolling(7, min_periods=3).mean()
df_cases_per_day

,Date_entry,Cases,Rolling_mean
0,2022-05-18,63,NaN
1,2022-05-19,40,NaN
2,2022-05-20,37,46.666667
3,2022-05-21,44,46.000000
4,2022-05-22,7,38.200000
5,2022-05-23,67,43.000000
6,2022-05-24,51,44.142857
7,2022-05-25,38,40.571429
8,2022-05-26,95,48.428571
9,2022-05-27,58,51.428571


In [100]:
line = alt.Chart(df_cases_per_day).mark_line().encode(
    x='Date_entry:T',
    y='Rolling_mean'
).properties(
    height = 200, 
    title = 'Anzahl Fälle nach Meldedatum'
)

In [101]:
bars = alt.Chart(df_roh[df_roh['Status'].isin(['confirmed', 'suspected'])]).mark_bar(size=10).encode(
    x=alt.X('Date_entry:T'  ),
    y=alt.Y('count()'), 
    color = alt.Color('Status'),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'Status',
      sort='ascending'
    )
).properties(
    height = 200, 
    title = 'Anzahl Fälle nach Meldedatum', 
)

In [102]:
bars + line

alt.LayerChart(...)

In [ ]:
df.rolling('2s', min_periods=1).sum()

## Debugging

* Replace ISO3 with country code that is not recognised
* Then copy name and use it to replace current country name at line 35 in py-skript.

### Länder finden für Pycountry

In [11]:

ISO3 = 'XXK' 

pycountry.countries.get(alpha_3=ISO3)

In [12]:
pycountry.countries.get(alpha_3='BOL').common_name

'Bolivia'

In [13]:
pycountry.countries.get(name='Mexico')

Country(alpha_2='MX', alpha_3='MEX', flag='🇲🇽', name='Mexico', numeric='484', official_name='United Mexican States')

In [ ]:
amendments = pd.read_csv('pycountry_amendments.csv')

In [ ]:
amendments

### Länder finden für Worldmap

In [10]:
ids[ids['ID'].str.contains('Kosovo')]

NameError: name 'ids' is not defined

In [37]:
df_worldmap[df_worldmap['ID'].str.contains('Malta')]

,ID,Wert
98,Malta,1.0


In [38]:
one_value = ['Mexico', 'Ecuador', 'Pakistan', 'Bolivia','Malta', 'Sudan', 'Austria', 'Finland']
df_worldmap[df_worldmap['ID'].isin(one_value)]

,ID,Wert
8,Austria,1.0
16,Bolivia,3.0
42,Ecuador,NaN
51,Finland,2.0
98,Malta,1.0
101,Mexico,1.0
120,Pakistan,1.0
148,Sudan,1.0
